In [1]:
from kliff.models import KIMModel
from kliff.dataset import Dataset
from kliff.ml.opt import OptimizerScipy
from scipy.optimize import minimize

In [20]:
dataset = Dataset(colabfit_database="colabfit_database", colabfit_dataset="jax_si")
dataset = Dataset("./")

2023-03-23 11:14:25.141 | INFO     | kliff.dataset.dataset:_read_colabfit:672 - 3716 configurations read from jax_si
2023-03-23 11:14:25.147 | INFO     | kliff.dataset.dataset:_read:628 - 4 configurations read from /home/amit/Documents/Presentations/Demos/SciML/example


In [21]:
!kim-api-collections-management install user "SW_StillingerWeber_1985_Si__MO_405512056662_005"

Item 'SW_StillingerWeber_1985_Si__MO_405512056662_005' already installed in collection 'user'.

Success!


In [22]:
model = KIMModel(model_name="SW_StillingerWeber_1985_Si__MO_405512056662_005")

In [5]:
model.set_opt_params(A=[[10.0]], B=[[0.5]])

In [25]:
model(dataset[0],compute_forces=False)

{'energy': -277.0156532289488, 'forces': None, 'stress': None}

### Explicit Optimization

In [26]:
param_to_optimize = model.parameters()
def loss_fn(params):
    for new_param, old_param in zip(params, param_to_optimize):
        model.copy_parameters(old_param, new_param)
    loss = 0
    for config in dataset:
        E = model(config)["energy"]
        loss += (config.energy - E)**2
    return loss

In [29]:
x0 = [10.0,5.5]
result = minimize(loss_fn, x0, method="L-BFGS-B", tol=1e-12, options={"maxiter":100})

In [30]:
result.x

array([10. ,  5.5])

### Implicit Optimization

In [10]:
opt = OptimizerScipy(model, param_to_optimize, dataset, optimizer="L-BFGS-B",target_property=["energy"],)
result_im = opt.minimize()

2023-03-23 08:57:49.907 | INFO     | kliff.ml.opt:minimize:120 - Starting with method L-BFGS-B


In [13]:
result_im.x

array([15.28515794,  0.60219939])